# load library

In [ ]:
#torch

from torch.nn import Parameter
import torch
import torch.optim
import torch.optim as optim
import torch.nn as nn
import torch.nn.parallel
import torch.nn.functional as F

import torchvision.models as models
import torchvision.transforms as transforms

import torch.utils.data as Data
import torch.backends.cudnn as cudnn

#numpy
import numpy as np

#pandas
import pandas as pd

#utility

import math
from urllib.request import urlretrieve

from PIL import Image
#from tqdm import tqdm
from tqdm.auto import tqdm
import random

import json
import os
import pickle
import shutil
import time

# initialize hyperparameter

In [ ]:
use_gpu = torch.cuda.is_available()

image_size = 224

batch_size = 64

num_classes = 23

workers = 4

max_epochs = 1

In [ ]:
encoding_label = {"classic": 0,
         "preppy":1, 
         "manish": 2,
         "tomboy":3, 
         "feminine": 4,
         "romantic": 5,
         "sexy": 6,
         "hippie": 7,
         "western": 8,
         "oriental": 9,
         "modern": 10,
         "sophisticated": 11,
         "avantgarde": 12,
         "country": 13,
         "resort": 14,
         "genderless": 15,
         "sporty": 16,
         "retro": 17,
         "hiphop": 18,
         "kitsch": 19,
         "punk": 20,
         "street": 21,
         "military":22
         }

# define metric

In [ ]:
#metric
def top_n_recall(preds, truths, n=1):
    best_n = np.argsort(preds, axis=1)[:,-n:]

    ts = []

    for i in range(len(truths[0])):
        
        if truths[0][i] == 1.0:
            
            ts.append(i)

    successes = 0
    s_img_id = []
    w_img_id = []

    for c in ts:
        if c in best_n:
            successes += 1
            s_img_id.append(c)
        else:
            w_img_id.append(c)

    return float(successes)/len(ts), s_img_id, w_img_id, best_n

# define utility

In [ ]:
class Warp(object):
    def __init__(self, size, interpolation=Image.BILINEAR):
        self.size = int(size)
        self.interpolation = interpolation

    def __call__(self, img):
        return img.resize((self.size, self.size), self.interpolation)

    def __str__(self):
        return self.__class__.__name__ + ' (size={size}, interpolation={interpolation})'.format(size=self.size,
                                                                                                interpolation=self.interpolation)
class MultiScaleCrop(object):

    def __init__(self, input_size, scales=None, max_distort=1, fix_crop=True, more_fix_crop=True):
        self.scales = scales if scales is not None else [1, 875, .75, .66]
        self.max_distort = max_distort
        self.fix_crop = fix_crop
        self.more_fix_crop = more_fix_crop
        self.input_size = input_size if not isinstance(input_size, int) else [input_size, input_size]
        self.interpolation = Image.BILINEAR

    def __call__(self, img):
        im_size = img.size
        crop_w, crop_h, offset_w, offset_h = self._sample_crop_size(im_size)
        crop_img_group = img.crop((offset_w, offset_h, offset_w + crop_w, offset_h + crop_h))
        ret_img_group = crop_img_group.resize((self.input_size[0], self.input_size[1]), self.interpolation)
        return ret_img_group

    def _sample_crop_size(self, im_size):
        image_w, image_h = im_size[0], im_size[1]

        # find a crop size
        base_size = min(image_w, image_h)
        crop_sizes = [int(base_size * x) for x in self.scales]
        crop_h = [self.input_size[1] if abs(x - self.input_size[1]) < 3 else x for x in crop_sizes]
        crop_w = [self.input_size[0] if abs(x - self.input_size[0]) < 3 else x for x in crop_sizes]

        pairs = []
        for i, h in enumerate(crop_h):
            for j, w in enumerate(crop_w):
                if abs(i - j) <= self.max_distort:
                    pairs.append((w, h))

        crop_pair = random.choice(pairs)
        if not self.fix_crop:
            w_offset = random.randint(0, image_w - crop_pair[0])
            h_offset = random.randint(0, image_h - crop_pair[1])
        else:
            w_offset, h_offset = self._sample_fix_offset(image_w, image_h, crop_pair[0], crop_pair[1])

        return crop_pair[0], crop_pair[1], w_offset, h_offset

    def _sample_fix_offset(self, image_w, image_h, crop_w, crop_h):
        offsets = self.fill_fix_offset(self.more_fix_crop, image_w, image_h, crop_w, crop_h)
        return random.choice(offsets)

    @staticmethod
    def fill_fix_offset(more_fix_crop, image_w, image_h, crop_w, crop_h):
        w_step = (image_w - crop_w) // 4
        h_step = (image_h - crop_h) // 4

        ret = list()
        ret.append((0, 0))  # upper left
        ret.append((4 * w_step, 0))  # upper right
        ret.append((0, 4 * h_step))  # lower left
        ret.append((4 * w_step, 4 * h_step))  # lower right
        ret.append((2 * w_step, 2 * h_step))  # center

        if more_fix_crop:
            ret.append((0, 2 * h_step))  # center left
            ret.append((4 * w_step, 2 * h_step))  # center right
            ret.append((2 * w_step, 4 * h_step))  # lower center
            ret.append((2 * w_step, 0 * h_step))  # upper center

            ret.append((1 * w_step, 1 * h_step))  # upper left quarter
            ret.append((3 * w_step, 1 * h_step))  # upper right quarter
            ret.append((1 * w_step, 3 * h_step))  # lower left quarter
            ret.append((3 * w_step, 3 * h_step))  # lower righ quarter

        return ret


    def __str__(self):
        return self.__class__.__name__


def download_url(url, destination=None, progress_bar=True):
    """Download a URL to a local file.

    Parameters
    ----------
    url : str
        The URL to download.
    destination : str, None
        The destination of the file. If None is given the file is saved to a temporary directory.
    progress_bar : bool
        Whether to show a command-line progress bar while downloading.

    Returns
    -------
    filename : str
        The location of the downloaded file.

    Notes
    -----
    Progress bar use/example adapted from tqdm documentation: https://github.com/tqdm/tqdm
    """

    def my_hook(t):
        last_b = [0]

        def inner(b=1, bsize=1, tsize=None):
            if tsize is not None:
                t.total = tsize
            if b > 0:
                t.update((b - last_b[0]) * bsize)
            last_b[0] = b

        return inner

    if progress_bar:
        with tqdm(unit='B', unit_scale=True, miniters=1, desc=url.split('/')[-1]) as t:
            filename, _ = urlretrieve(url, filename=destination, reporthook=my_hook(t))
    else:
        filename, _ = urlretrieve(url, filename=destination)


class AveragePrecisionMeter(object):
    """
    The APMeter measures the average precision per class.
    The APMeter is designed to operate on `NxK` Tensors `output` and
    `target`, and optionally a `Nx1` Tensor weight where (1) the `output`
    contains model output scores for `N` examples and `K` classes that ought to
    be higher when the model is more convinced that the example should be
    positively labeled, and smaller when the model believes the example should
    be negatively labeled (for instance, the output of a sigmoid function); (2)
    the `target` contains only values 0 (for negative examples) and 1
    (for positive examples); and (3) the `weight` ( > 0) represents weight for
    each sample.
    """

    def __init__(self, difficult_examples=False):
        super(AveragePrecisionMeter, self).__init__()
        self.reset()
        self.difficult_examples = difficult_examples

    def reset(self):
        """Resets the meter with empty member variables"""
        self.scores = torch.FloatTensor(torch.FloatStorage())
        self.targets = torch.LongTensor(torch.LongStorage())

    def add(self, output, target):
        """
        Args:
            output (Tensor): NxK tensor that for each of the N examples
                indicates the probability of the example belonging to each of
                the K classes, according to the model. The probabilities should
                sum to one over all classes
            target (Tensor): binary NxK tensort that encodes which of the K
                classes are associated with the N-th input
                    (eg: a row [0, 1, 0, 1] indicates that the example is
                         associated with classes 2 and 4)
            weight (optional, Tensor): Nx1 tensor representing the weight for
                each example (each weight > 0)
        """
        if not torch.is_tensor(output):
            output = torch.from_numpy(output)
        if not torch.is_tensor(target):
            target = torch.from_numpy(target)

        if output.dim() == 1:
            output = output.view(-1, 1)
        else:
            assert output.dim() == 2, \
                'wrong output size (should be 1D or 2D with one column \
                per class)'
        if target.dim() == 1:
            target = target.view(-1, 1)
        else:
            assert target.dim() == 2, \
                'wrong target size (should be 1D or 2D with one column \
                per class)'
        if self.scores.numel() > 0:
            assert target.size(1) == self.targets.size(1), \
                'dimensions for output should match previously added examples.'

        # make sure storage is of sufficient size
        if self.scores.storage().size() < self.scores.numel() + output.numel():
            new_size = math.ceil(self.scores.storage().size() * 1.5)
            self.scores.storage().resize_(int(new_size + output.numel()))
            self.targets.storage().resize_(int(new_size + output.numel()))

        # store scores and targets
        offset = self.scores.size(0) if self.scores.dim() > 0 else 0
        self.scores.resize_(offset + output.size(0), output.size(1))
        self.targets.resize_(offset + target.size(0), target.size(1))
        self.scores.narrow(0, offset, output.size(0)).copy_(output)
        self.targets.narrow(0, offset, target.size(0)).copy_(target)

    def value(self):
        """Returns the model's average precision for each class
        Return:
            ap (FloatTensor): 1xK tensor, with avg precision for each class k
        """

        if self.scores.numel() == 0:
            return 0
        ap = torch.zeros(self.scores.size(1))
        rg = torch.arange(1, self.scores.size(0)).float()
        # compute average precision for each class
        for k in range(self.scores.size(1)):
            # sort scores
            scores = self.scores[:, k]
            targets = self.targets[:, k]
            # compute average precision
            ap[k] = AveragePrecisionMeter.average_precision(scores, targets, self.difficult_examples)
        return ap

    @staticmethod
    def average_precision(output, target, difficult_examples=True):

        # sort examples
        sorted, indices = torch.sort(output, dim=0, descending=True)

        # Computes prec@i
        pos_count = 0.
        total_count = 0.
        precision_at_i = 0.
        for i in indices:
            label = target[i]
            if difficult_examples and label == 0:
                continue
            if label == 1:
                pos_count += 1
            total_count += 1
            if label == 1:
                precision_at_i += pos_count / total_count
        precision_at_i /= pos_count
        return precision_at_i

    def overall(self):
        if self.scores.numel() == 0:
            return 0
        scores = self.scores.cpu().numpy()
        targets = self.targets.cpu().numpy()
        targets[targets == -1] = 0
        return self.evaluation(scores, targets)

    def overall_topk(self, k):
        targets = self.targets.cpu().numpy()
        targets[targets == -1] = 0
        n, c = self.scores.size()
        scores = np.zeros((n, c)) - 1
        index = self.scores.topk(k, 1, True, True)[1].cpu().numpy()
        tmp = self.scores.cpu().numpy()
        for i in range(n):
            for ind in index[i]:
                scores[i, ind] = 1 if tmp[i, ind] >= 0 else -1
        return self.evaluation(scores, targets)


    def evaluation(self, scores_, targets_):
        n, n_class = scores_.shape
        Nc, Np, Ng = np.zeros(n_class), np.zeros(n_class), np.zeros(n_class)
        for k in range(n_class):
            scores = scores_[:, k]
            targets = targets_[:, k]
            targets[targets == -1] = 0
            Ng[k] = np.sum(targets == 1)
            Np[k] = np.sum(scores >= 0)
            Nc[k] = np.sum(targets * (scores >= 0))
        Np[Np == 0] = 1
        OP = np.sum(Nc) / np.sum(Np)
        OR = np.sum(Nc) / np.sum(Ng)
        OF1 = (2 * OP * OR) / (OP + OR)

        CP = np.sum(Nc / Np) / n_class
        CR = np.sum(Nc / Ng) / n_class
        CF1 = (2 * CP * CR) / (CP + CR)
        return OP, OR, OF1, CP, CR, CF1

def top_n_recall(preds, truths, n=1):
    best_n = np.argsort(preds, axis=1)[:,-n:]
    ts = np.argmax(truths, axis=1)
    successes = 0
    s_img_id = []
    w_img_id = []
    for i in range(ts.shape[0]):
        if ts[i] in best_n[i,:]:
            successes += 1
            s_img_id.append(i)
        else:
            w_img_id.append(i)

    return float(successes)/ts.shape[0], s_img_id, w_img_id, best_n

def gen_A(num_classes, t, adj_file):
    import pickle
    result = pickle.load(open(adj_file, 'rb'))
    _adj = result['adj']
    _nums = result['nums']
    _nums = _nums[:, np.newaxis]
    _adj = _adj / _nums
    _adj[_adj < t] = 0
    _adj[_adj >= t] = 1
    _adj = _adj * 0.25 / (_adj.sum(0, keepdims=True) + 1e-6)
    _adj = _adj + np.identity(num_classes, np.int)
    return _adj

def gen_adj(A):
    D = torch.pow(A.sum(1).float(), -0.5)
    D = torch.diag(D)
    adj = torch.matmul(torch.matmul(A, D).t(), D)
    return adj

# create raw data

In [ ]:
nums = [0]*23
adj = [[0]*23 for _ in range(23)]
train_data_list = []

In [ ]:
with open("/content/drive/MyDrive/singledata/패션소품/watch.json", 'r') as file:
    data = json.load(file)

In [ ]:
for d in data:
    
    print(d['imageUrl'])
    break


https://image.brandi.me/cproduct/2021/04/28/SB000000000008366784_1619597393_image1_S.jpeg


In [ ]:
data

In [ ]:
for d in tqdm(data):
    
    train_data = {}

    one = d['style_vector']['1']
    two = d['style_vector']['2']
    three = d['style_vector']['3']

    labels = []

    if one['score'] > (two['score'] + three['score']):
        
        labels.append(encoding_label[one['style']])

        A = labels[0]

        nums[A] += 1

    else:
        
        labels.append(encoding_label[one['style']])
        labels.append(encoding_label[two['style']])

        A = labels[0]
        B = labels[1]

        nums[A] += 1
        nums[B] += 1

        adj[A][B] += 1
        adj[B][A] += 1
    
    train_data['labels'] = labels
    train_data['file_name'] = d['imageUrl']

    train_data_list.append(train_data)

len(train_data_list)

  0%|          | 0/100 [00:00<?, ?it/s]

4675

In [ ]:
train_data_list

In [ ]:
nums

[121,
 28,
 0,
 0,
 15,
 126,
 317,
 91,
 0,
 0,
 396,
 154,
 34,
 71,
 0,
 190,
 46,
 23,
 0,
 135,
 1,
 2503,
 75]

In [ ]:
adj_dict = {}

adj_dict["nums"] = np.array(nums)
adj_dict["adj"] = np.array(adj)

In [ ]:
with open("/content/drive/MyDrive/pickle/new_single_data_annotation_v2.pkl","wb") as fw:
    pickle.dump(train_data_list, fw)

In [ ]:
with open("/content/drive/MyDrive/pickle/single_adj_file.pkl","wb") as fw:
    pickle.dump(adj_dict, fw)

# define model

In [ ]:
class GraphConvolution(nn.Module):

    def __init__(self, in_features, out_features, bias=False):
        super(GraphConvolution, self).__init__()
        self.in_features = in_features
        self.out_features = out_features
        self.weight = Parameter(torch.Tensor(in_features, out_features))
        if bias:
            self.bias = Parameter(torch.Tensor(1, 1, out_features))
        else:
            self.register_parameter('bias', None)
        self.reset_parameters()

    def reset_parameters(self):
        stdv = 1. / math.sqrt(self.weight.size(1))
        self.weight.data.uniform_(-stdv, stdv)
        if self.bias is not None:
            self.bias.data.uniform_(-stdv, stdv)

    def forward(self, input, adj):
        support = torch.matmul(input, self.weight)
        output = torch.matmul(adj, support)
        if self.bias is not None:
            return output + self.bias
        else:
            return output

    def __repr__(self):
        return self.__class__.__name__ + ' (' \
               + str(self.in_features) + ' -> ' \
               + str(self.out_features) + ')'


class GCNResnet(nn.Module):
    def __init__(self, model, num_classes, in_channel=300, t=0, adj_file=None):
        super(GCNResnet, self).__init__()
        self.features = nn.Sequential(
            model.conv1,
            model.bn1,
            model.relu,
            model.maxpool,
            model.layer1,
            model.layer2,
            model.layer3,
            model.layer4,
        )
        self.num_classes = num_classes
        # self.pooling = nn.MaxPool2d(14, 14)
        self.pooling = nn.MaxPool2d(7, 7)

        self.gc1 = GraphConvolution(in_channel, 1024)
        self.gc2 = GraphConvolution(1024, 2048)
        self.relu = nn.LeakyReLU(0.2)

        _adj = gen_A(num_classes, t, adj_file)
        self.A = Parameter(torch.from_numpy(_adj).float())
        #         # image normalization
        self.image_normalization_mean = [0.485, 0.456, 0.406]
        self.image_normalization_std = [0.229, 0.224, 0.225]

    def forward(self, feature, inp):
        feature = self.features(feature)
        feature = self.pooling(feature)
        feature = feature.view(feature.size(0), -1)
        inp = inp[0]
        adj = gen_adj(self.A).detach()
        x = self.gc1(inp, adj)
        x = self.relu(x)
        x = self.gc2(x, adj)
        x = x.transpose(0, 1)
        x = torch.matmul(feature, x)
        return x

    def get_config_optim(self, lr, lrp):
        return [
                {'params': self.features.parameters(), 'lr': lr * lrp},
                {'params': self.gc1.parameters(), 'lr': lr},
                {'params': self.gc2.parameters(), 'lr': lr},
                ]


def gcn_resnet101(num_classes, t, pretrained=True, adj_file=None, in_channel=300):
    model = models.resnet101(pretrained=pretrained)
    return GCNResnet(model, num_classes, t=t, adj_file=adj_file, in_channel=in_channel)

In [ ]:
#load ImageNet pretrained model

adj = "/content/drive/MyDrive/pickle/single_adj_file.pkl"

model = gcn_resnet101(num_classes=num_classes, t=0.03, pretrained = True, adj_file=adj, in_channel = 300)

/usr/local/lib/python3.9/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.9/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet101_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet101_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet101-63fe2227.pth" to /root/.cache/torch/hub/checkpoints/resnet101-63fe2227.pth


  0%|          | 0.00/171M [00:00<?, ?B/s]

<ipython-input-5-ac00a1f67dd8>:303: RuntimeWarning: invalid value encountered in true_divide
  _adj = _adj / _nums
<ipython-input-5-ac00a1f67dd8>:307: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  _adj = _adj + np.identity(num_classes, np.int)


In [ ]:
#load best model

resume = "/content/drive/MyDrive/singlemodel_save/single_best_model_v2.pth"

#checkpoint = torch.load(resume)
checkpoint = torch.load(resume,map_location=torch.device("cpu"))

model.load_state_dict(checkpoint)

<All keys matched successfully>

# create dataset

In [ ]:
#transform

normalize = transforms.Normalize(mean = model.image_normalization_mean, std = model.image_normalization_std)

train_transform = transforms.Compose([
    MultiScaleCrop(image_size, scales = (1.0,0.875,0.75, 0.66,0.5), max_distort = 2),
    #transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    normalize,
])

val_transform = transforms.Compose([
    Warp(image_size),
    transforms.ToTensor(),
    normalize,
])

In [ ]:
## Load word2vec
with open("/content/drive/MyDrive/pickle/custom_word2vec.pkl","rb") as fr:
    word2vec = pickle.load(fr)
print(word2vec)

[[ 0.2890625   0.06982422  0.16210938 ... -0.12451172  0.02148438
  -0.04370117]
 [ 0.03930664  0.10595703 -0.18359375 ... -0.21484375  0.16113281
   0.36132812]
 [-0.14550781  0.03637695  0.05175781 ... -0.0625     -0.07910156
   0.02819824]
 ...
 [ 0.05224609 -0.03808594  0.20410156 ... -0.265625    0.09570312
   0.14355469]
 [ 0.09814453  0.07568359  0.03088379 ... -0.04833984  0.21289062
  -0.13867188]
 [-0.03442383  0.21679688 -0.09472656 ... -0.04003906  0.15527344
   0.10009766]]


In [ ]:
with open("/content/drive/MyDrive/pickle/new_single_data_annotation_v2.pkl","rb") as fr:
    train_data_list = pickle.load(fr)

len(train_data_list)

4675

In [ ]:
class SingleDataset(Data.Dataset):
    def __init__(self, data_list, word2vec, transform, num_classes=23):
        self.img_list = data_list
        self.transform = transform
        self.num_classes = num_classes
        self.inp = torch.tensor(word2vec)

    def __len__(self):
        return len(self.img_list)

    def __getitem__(self, index):
        item = self.img_list[index]
        return self.get(item)

    def get(self, item):
        filename = item['file_name']
        labels = sorted(item['labels'])

        img = Image.open(urlretrieve(filename)[0]).convert("RGB")

        if self.transform is not None:
            img = self.transform(img)
            
        target = np.zeros(self.num_classes, np.float32) - 1
        target[labels] = 1
        return (img, filename, self.inp), torch.tensor(target)

In [ ]:
#dataset

train_dataset = SingleDataset(train_data_list, word2vec, train_transform, 23)



In [ ]:
#data loader

train_loader = torch.utils.data.DataLoader(train_dataset,
                                            batch_size=batch_size, shuffle=True,
                                            num_workers=workers)

#criterion & optimizer

In [ ]:
class BCEWithLogitsLossWeighted(torch.nn.Module):
    def __init__(self, weight, *args, **kwargs):
        super().__init__()
        # Notice none reduction
        self.bce = torch.nn.BCEWithLogitsLoss(*args, **kwargs, reduction="none")
        self.weight = weight

    def forward(self, logits, labels):
        loss = self.bce(logits, labels)
        binary_labels = labels.bool()
        loss[binary_labels] *= labels[binary_labels] * self.weight
        # Or any other reduction
        return torch.mean(loss)

In [ ]:
#criterion = nn.MultiLabelSoftMarginLoss()

criterion = BCEWithLogitsLossWeighted(23)

optimizer = torch.optim.Adam(model.parameters() , lr=0.0005)

scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=50, eta_min=0)

In [ ]:
model

# self-supervised training

In [ ]:
train_dataset[0]

((tensor([[[-0.5424, -0.5767, -0.5767,  ..., -0.8335, -0.5253, -0.2684],
           [-0.5424, -0.5424, -0.5938,  ..., -0.7479, -0.3541, -0.4054],
           [-0.5767, -0.5596, -0.5767,  ..., -0.7993, -0.4397, -0.6452],
           ...,
           [-1.7240, -1.7240, -1.7069,  ..., -0.2342, -0.3027, -0.2171],
           [-1.7240, -1.7240, -1.6727,  ..., -0.2856, -0.2513, -0.1314],
           [-1.7069, -1.7069, -1.6898,  ..., -0.2171, -0.1657, -0.0458]],
  
          [[-0.9153, -0.9678, -0.9853,  ..., -1.1779, -1.0203, -0.8277],
           [-0.9153, -0.9153, -1.0028,  ..., -1.0553, -0.7927, -0.9153],
           [-0.9503, -0.9503, -0.9853,  ..., -1.0903, -0.8277, -1.0903],
           ...,
           [-1.8782, -1.8782, -1.8431,  ..., -0.8978, -0.9678, -0.8803],
           [-1.8957, -1.8957, -1.8431,  ..., -0.9503, -0.9153, -0.7927],
           [-1.8782, -1.8782, -1.8606,  ..., -0.8978, -0.8452, -0.7227]],
  
          [[-1.0201, -1.0724, -1.1073,  ..., -1.2467, -1.1596, -0.9853],
           

In [ ]:
save_path = '/content/drive/MyDrive/singlemodel_save/single_best_model_v2.pth'

In [ ]:
#acclerating gpu
torch.backends.cudnn.benchmark = True

model = model.cuda()

# switch to train mode
model.train()

max_epochs = 5

for epoch in range(max_epochs):
    
    epoch_loss = 0.0
    epoch_recall = 0.0

    print(f"epoch: {epoch}")
    print("-------------------------------------------------------------")

    for i,(input,target) in enumerate(tqdm(train_loader),start = 1):

        target_gt = target.clone()

        target[target == 0] = 1
        target[target == -1] = 0

        feature,_,input = input

        feature_var = torch.autograd.Variable(feature).float()
        target_var = torch.autograd.Variable(target).float()
        inp_var = torch.autograd.Variable(input).float().detach()

        feature_var = feature_var.cuda()
        inp_var = inp_var.cuda()
        target_var = target_var.cuda()

        output = model(feature_var,inp_var)

        loss = criterion(output,target_var)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        epoch_loss += loss.item()

        pred = output.cpu().detach().numpy()
        real = target.cpu().detach().numpy()
        
        epoch_recall += top_n_recall(pred,real,3)[0]

        # if i % 50 == 0:
            
        #     print(f"loss : {epoch_loss/i:.4f}")
        #     print(f"recall : {epoch_recall/i:.4f}")

        #     torch.save(model.state_dict(), save_path)

        #     #scheduler.step()

    epoch_loss = epoch_loss/len(train_loader)
    epoch_recall = epoch_recall/len(train_loader)

    print(f"epoch loss : {epoch_loss:.4f}")
    print(f"recall : {epoch_recall:.4f}")

    torch.save(model.state_dict(), save_path)
    
    print("-------------------------------------------------------------")
    print()

epoch: 0
-------------------------------------------------------------


  0%|          | 0/74 [00:00<?, ?it/s]

epoch loss : 0.3197
recall : 0.9559
-------------------------------------------------------------

epoch: 1
-------------------------------------------------------------


  0%|          | 0/74 [00:00<?, ?it/s]

epoch loss : 0.3142
recall : 0.9525
-------------------------------------------------------------

epoch: 2
-------------------------------------------------------------


...

epoch loss : 0.2780
recall : 0.9609
-------------------------------------------------------------

epoch: 3
-------------------------------------------------------------


  0%|          | 0/74 [00:00<?, ?it/s]

epoch loss : 0.2819
recall : 0.9636
-------------------------------------------------------------

epoch: 4
-------------------------------------------------------------


  0%|          | 0/74 [00:00<?, ?it/s]

epoch loss : 0.3282
recall : 0.9499
-------------------------------------------------------------



# relabeling data

In [ ]:
inp = torch.tensor(word2vec).unsqueeze(0)

In [ ]:
change_class_style = {0: "classic",
         1: "preppy", 
         2: "manish",
         3: "tomboy", 
         4: "feminine",
         5: "romantic",
         6: "sexy",
         7: "hippie",
         8: "western",
         9: "oriental",
         10: "modern",
         11: "sophisticated",
         12: "avantgarde",
         13: "country",
         14: "resort",
         15: "genderless",
         16: "sporty",
         17: "retro",
         18: "hiphop",
         19: "kitsch",
         20: "punk",
         21: "street",
         22: "military"
         }

In [ ]:
root_path = "/content/drive/MyDrive/singledata/패션소품/"

json_list = os.listdir("/content/drive/MyDrive/singledata/패션소품/")

for filename in tqdm(json_list):

    with open(root_path + filename, 'r') as file:
        data = json.load(file)

    url_list = []

    for d in data:
        
        url_list.append(d['imageUrl'])

    model.eval()

    for j,url in tqdm(enumerate(url_list)):

        img = Image.open(urlretrieve(url)[0]).convert("RGB")

        if val_transform is not None:
        
            img = val_transform(img)
        
        img = img.unsqueeze(0)

        feature_var = torch.autograd.Variable(img).float()
        inp_var = torch.autograd.Variable(inp).float().detach()  # one hot

        output = model(feature_var, inp_var)
        percentage_output = F.softmax(output, dim = 1)

        pred = output.cpu().detach().numpy()

        sorted_pred = np.argsort(pred,axis = 1)

        result = {}

        for i in range(22,-1,-1):
            
            result[23-i] = {
                "style":change_class_style[sorted_pred[0][i]],
                "score":round(percentage_output[0][sorted_pred[0][i]].item()*100,4)
            }
        
        data[j]['style_vector'] = result
    
    with open(root_path + filename, 'w', encoding='utf-8') as file:
        
        json.dump(data, file)

  0%|          | 0/7 [00:00<?, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]